In [ ]:
! pip install langchain-mcp-adapters langgraph "langchain[anthropic]" langgraph-swarm httpx markdownify

In [9]:
# Imports
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient

from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

planner_prompt = """
<Task>
You will help plan the steps to implement a LangGraph application based on the user's request. 
</Task>

<Instructions>
1. Reflect on the user's request. 
2. Use the fetch_doc tool to read this llms.txt file: {llms_txt}
3. Identify documents that are relevant to the user's request.
4. Ask any follow-up questions to help refine the project scope and narrow the set of documents to be used for the project.
5. When the project scope is clear produce two things: 
- A short description that lays out the scope of the project 
- A list of relevant URLs to reference in order to implement the project
6. Finally, transfer to the research agent using the transfer_to_researcher_agent tool.
</Instructions>
"""

researcher_prompt = """
<Task>
You will perform research using provided URLs and use these to implement the solution to the user's request. 
</Task>

<Instructions>
1. Reflect on the project scope and provided URLs.
2. Use the fetch_doc tool to fetch and read each URL.
3. Use the information in these URLs to implement the solution to the user's request.
4. If you need further clarification or additional sources to implement the solution, then transfer to transfer_to_planner_agent.
</Instructions>
"""

# LLM
model = ChatAnthropic(model="claude-3-7-sonnet-latest")

# Handoff tools
transfer_to_planner_agent = create_handoff_tool(
    agent_name="planner_agent",
    description="Transfer the user to the planner_agent for clarifying questions related to the user's request."
)
transfer_to_researcher_agent = create_handoff_tool(
    agent_name="researcher_agent",
    description="Transfer the user to the researcher_agent to perform research and implement the solution to the user's request."
)

# LLMS.txt
llms_txt = "LangGraph:https://langchain-ai.github.io/langgraph/llms.txt"
planner_prompt_formatted = planner_prompt.format(llms_txt=llms_txt)


In [4]:
import httpx
from markdownify import markdownify

httpx_client = httpx.Client(follow_redirects=False, timeout=10)

def fetch_doc(url: str) -> str:
        """ Fetch a document from a URL and return the markdownified text. 
        Args:
            url (str): The URL of the document to fetch.
        Returns:
            str: The markdownified text of the document.
        """
   
        try:
            response = httpx_client.get(url, timeout=10)
            response.raise_for_status()
            return markdownify(response.text)
        except (httpx.HTTPStatusError, httpx.RequestError) as e:
            return f"Encountered an HTTP error: {str(e)}"


In [10]:
from langgraph.checkpoint.memory import InMemorySaver

planner_agent = create_react_agent(model,
                                   prompt=planner_prompt_formatted, 
                                   tools=[fetch_doc,transfer_to_researcher_agent],
                                   name="planner_agent") 

# Researcher agent
researcher_agent = create_react_agent(model, 
                                    prompt=researcher_prompt, 
                                    tools=[fetch_doc,transfer_to_researcher_agent],
                                    name="researcher_agent") 

# Swarm
checkpointer = InMemorySaver()
agent_swarm = create_swarm([planner_agent, researcher_agent], default_active_agent="planner_agent")
app = agent_swarm.compile(checkpointer=checkpointer)


In [ ]:
# Input 
request =  "Create a LangGraph application that is a prompt chain: it takes a topic from a user, generates a joke, and checks if the joke has a punchline."
messages = {"role": "user", "content": request}
response = app.invoke({"messages": messages})

### Use MCP 

In [ ]:
from langchain_mcp_adapters.client import StdioServerParameters

server_params = StdioServerParameters(
    command="uvx",
    args=[
            "--from",
            "mcpdoc",
            "mcpdoc",
            "--urls",
            "LangGraph:https://langchain-ai.github.io/langgraph/llms.txt",
            "--transport",
            "stdio",
            "--port",
            "8081",
            "--host",
            "localhost"
            ],
)

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        planner_agent = create_react_agent(model,
                                        prompt=planner_prompt, 
                                        tools=tools.append(transfer_to_researcher_agent),
                                        name="planner_agent") 

        # Researcher agent
        researcher_agent = create_react_agent(model, 
                                            prompt=researcher_prompt, 
                                            tools=tools.append(transfer_to_planner_agent),
                                            name="researcher_agent") 

        # Swarm
        agent_swarm = create_swarm([planner_agent, researcher_agent], default_active_agent="planner_agent")

        # app = agent_swarm.compile(config_schema=Configuration)
        agent = agent_swarm.compile()
        response = await agent.ainvoke({"messages": messages})


In [ ]:
for m in agent_response['messages']:
    m.pretty_print()

In [ ]:

# Research and planning tools 
async with MultiServerMCPClient(
    {
        "planning-server": {
            "command": "uvx",
            "args": [
            "--from",
            "mcpdoc",
            "mcpdoc",
            "--urls",
            llms_txt_urls,
            "--transport",
            "stdio",
            "--port",
            "8081",
            "--host",
            "localhost"
            ],
            "transport": "stdio",
        }
    }

) as client:
    # Planner agent
    planner_agent = create_react_agent(model,
                                       prompt=planner_prompt, 
                                       tools=client.server_name_to_tools["planning-server"].append(transfer_to_researcher_agent),
                                       name="planner_agent") 

    # Researcher agent
    researcher_agent = create_react_agent(model, 
                                          prompt=researcher_prompt, 
                                          tools=client.server_name_to_tools["planning-server"].append(transfer_to_planner_agent),
                                          name="researcher_agent") 

    # Swarm
    agent_swarm = create_swarm([planner_agent, researcher_agent], default_active_agent="planner_agent")

    # app = agent_swarm.compile(config_schema=Configuration)
    agent = agent_swarm.compile()
    response = await agent.ainvoke({"messages": messages})

In [ ]:
{
        "research-server": {
            "command": "npx",
            "args": ["@playwright/mcp"],
            "transport": "stdio",
            "env": {
                "PATH": "/Users/rlm/.cursor/extensions/ms-python.python-2024.12.3-darwin-arm64/python_files/deactivate/zsh:/Users/rlm/Desktop/Code/langgraph-swarm/.venv/bin:/Users/rlm/.bun/bin:/Users/rlm/.poetry/bin:/Users/rlm/Library/Python/3.13/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/Users/rlm/.cargo/bin:/Users/rlm/miniforge3/condabin:/Users/rlm/.local/bin"
            }
        },
        "planning-server": {
            "command": "uvx",
            "args": [
            "--from",
            "mcpdoc",
            "mcpdoc",
            "--urls",
            llms_txt_urls,
            "--transport",
            "stdio",
            "--port",
            "8081",
            "--host",
            "localhost"
            ],
            "transport": "stdio",
        }
    }